# Dummy Agent Library

In this simple example, **we're going to code an Agent from scratch**.

This notebook is part of the <a href="https://www.hf.co/learn/agents-course">Hugging Face Agents Course</a>, a free Course from beginner to expert, where you learn to build Agents.

<img src="https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png" alt="Agent Course"/>

In [1]:
import os

# Ask the user to enter their Hugging Face token
hf_token = input("Enter your Hugging Face token: ")

# Store it as an environment variable for future use
os.environ["HF_TOKEN"] = hf_token

print("Token has been set successfully!")

Token has been set successfully!


In [2]:
!pip install -q huggingface_hub

## Serverless API

In the Hugging Face ecosystem, there is a convenient feature called Serverless API that allows you to easily run inference on many models. There's no installation or deployment required.

To run this notebook, **you need a Hugging Face token** that you can get from https://hf.co/settings/tokens. A "Read" token type is sufficient. 
- If you are running this notebook on Google Colab, you can set it up in the "settings" tab under "secrets". Make sure to call it "HF_TOKEN" and restart the session to load the environment variable (Runtime -> Restart session).
- If you are running this notebook locally, you can set it up as an [environment variable](https://huggingface.co/docs/huggingface_hub/en/package_reference/environment_variables). Make sure you restart the kernel after installing or updating huggingface_hub. You can update huggingface_hub by modifying the above `!pip install -q huggingface_hub -U`

You also need to request access to [the Meta Llama models](https://huggingface.co/meta-llama), select [Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct) if you haven't done it click on Expand to review and access and fill the form. Approval usually takes up to an hour.

## Serverless API

In the Hugging Face ecosystem, there is a convenient feature called Serverless API that allows you to easily run inference on many models. There's no installation or deployment required.

To run this notebook, **you need a Hugging Face token** that you can get from https://hf.co/settings/tokens. A "Read" token type is sufficient. 
- If you are running this notebook on Google Colab, you can set it up in the "settings" tab under "secrets". Make sure to call it "HF_TOKEN" and restart the session to load the environment variable (Runtime -> Restart session).
- If you are running this notebook locally, you can set it up as an [environment variable](https://huggingface.co/docs/huggingface_hub/en/package_reference/environment_variables). Make sure you restart the kernel after installing or updating huggingface_hub. You can update huggingface_hub by modifying the above `!pip install -q huggingface_hub -U`

You also need to request access to [the Meta Llama models](https://huggingface.co/meta-llama), select [Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct) if you haven't done it click on Expand to review and access and fill the form. Approval usually takes up to an hour.

In [7]:
import os
from huggingface_hub import InferenceClient

HF_TOKEN = os.environ.get("HF_TOKEN")



client = InferenceClient("meta-llama/Llama-4-Scout-17B-16E-Instruct")
# if the outputs for next cells are wrong, the free model may be overloaded. You can also use this public endpoint that contains Llama-3.2-3B-Instruct
#client = InferenceClient("https://jc26mwg228mkj8dw.us-east-1.aws.endpoints.huggingface.cloud")

In [9]:
# As seen in the LLM section, if we just do decoding, **the model will only stop when it predicts an EOS token**, 
# and this does not happen here because this is a conversational (chat) model and we didn't apply the chat template it expects.
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of france is"},
    ],
    stream=False,
    max_tokens=100,
)

print(output.choices[0].message.content)

Paris.


As seen in the LLM section, if we just do decoding, **the model will only stop when it predicts an EOS token**, and this does not happen here because this is a conversational (chat) model and **we didn't apply the chat template it expects**.

If we now add the special tokens related to the <a href="https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct">Llama-3.2-3B-Instruct model</a> that we're using, the behavior changes and it now produces the expected EOS.

In [ ]:
# If we now add the special tokens related to Llama4 model, the behaviour changes and is now the expected one.
prompt="""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

The capital of france is<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of france is"},
    ],
    stream=False,
    max_tokens=100,
)

print(output.choices[0].message.content)


Paris.


Using the "chat" method is a much more convenient and reliable way to apply chat templates:

In [12]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of france is"},
    ],
    stream=False,
    max_tokens=100,
)

print(output.choices[0].message.content)

Paris.


The chat method is the RECOMMENDED method to use in order to ensure a **smooth transition between models but since this notebook is only educational**, we will keep using the "text_generation" method to understand the details.


## Dummy Agent

In the previous sections, we saw that the **core of an agent library is to append information in the system prompt**.

This system prompt is a bit more complex than the one we saw earlier, but it already contains:

1. **Information about the tools**
2. **Cycle instructions** (Thought → Action → Observation)

In [13]:
# This system prompt is a bit more complex and actually contains the function description already appended.
# Here we suppose that the textual description of the tools have already been appended
SYSTEM_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args: {{"location": {{"type": "string"}}}}
example use :
```
{{
  "action": "get_weather",
  "action_input": {"location": "New York"}
}}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:
```
$JSON_BLOB
```
Observation: the result of the action. This Observation is unique, complete, and the source of truth.
... (this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $JSON_BLOB must be formatted as markdown and only use a SINGLE action at a time.)

You must always end your output with the following format:

Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. """
